<a href="https://colab.research.google.com/github/tararajagopalan/WebsiteQueryingProject/blob/main/project_querySearch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#INSTALLING PYMILVUS

!pip install -U pymilvus
!pip install --upgrade pymilvus
!pip install "pymilvus[model]"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.3/254.3 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.3/55.3 MB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.4/57.4 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 82.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 7.5 MB/s eta 0:00:00


In [ ]:
#SETTING UP VECTOR DATA BASE: creating a database on the mounted google drive
#sets up data base in the milvus_demo.db file on google drive
from pymilvus import MilvusClient

milvus_client = MilvusClient("/content/drive/MyDrive/project_work.db")

In [ ]:
milvus_client.describe_collection("project_collection")

{'collection_name': 'project_collection',
 'auto_id': True,
 'num_shards': 0,
 'description': '',
 'fields': [{'field_id': 100,
   'name': 'id',
   'description': '',
   'type': <DataType.INT64: 5>,
   'params': {},
   'auto_id': True,
   'is_primary': True},
  {'field_id': 101,
   'name': 'text',
   'description': '',
   'type': <DataType.VARCHAR: 21>,
   'params': {'max_length': 2048}},
  {'field_id': 102,
   'name': 'embedding',
   'description': '',
   'type': <DataType.FLOAT_VECTOR: 101>,
   'params': {'dim': 768}},
  {'field_id': 103,
   'name': 'FilePath',
   'description': '',
   'type': <DataType.VARCHAR: 21>,
   'params': {'max_length': 80}},
  {'field_id': 104,
   'name': 'FileExtension',
   'description': '',
   'type': <DataType.VARCHAR: 21>,
   'params': {'max_length': 80}}],
 'functions': [],
 'aliases': [],
 'collection_id': 0,
 'consistency_level': 0,
 'properties': {},
 'num_partitions': 0,
 'enable_dynamic_field': False}

In [ ]:
from pymilvus import model

embedding_fn = model.DefaultEmbeddingFunction()

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/465 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/827 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/760k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/245 [00:00<?, ?B/s]

model.onnx:   0%|          | 0.00/46.9M [00:00<?, ?B/s]

In [ ]:
query_vectors = embedding_fn.encode_queries(["What is a Unicorn?"])


search_params = {
    "metric_type": "L2",          # or "IP" depending on your index
    "params": {"nprobe": 10}
}


In [ ]:
print(len(query_vectors[0]))

milvus_client.describe_collection("project_collection")

768


{'collection_name': 'project_collection',
 'auto_id': True,
 'num_shards': 0,
 'description': '',
 'fields': [{'field_id': 100,
   'name': 'id',
   'description': '',
   'type': <DataType.INT64: 5>,
   'params': {},
   'auto_id': True,
   'is_primary': True},
  {'field_id': 101,
   'name': 'text',
   'description': '',
   'type': <DataType.VARCHAR: 21>,
   'params': {'max_length': 2048}},
  {'field_id': 102,
   'name': 'embedding',
   'description': '',
   'type': <DataType.FLOAT_VECTOR: 101>,
   'params': {'dim': 768}},
  {'field_id': 103,
   'name': 'FilePath',
   'description': '',
   'type': <DataType.VARCHAR: 21>,
   'params': {'max_length': 80}},
  {'field_id': 104,
   'name': 'FileExtension',
   'description': '',
   'type': <DataType.VARCHAR: 21>,
   'params': {'max_length': 80}}],
 'functions': [],
 'aliases': [],
 'collection_id': 0,
 'consistency_level': 0,
 'properties': {},
 'num_partitions': 0,
 'enable_dynamic_field': False}

In [ ]:
search_params = {
    "metric_type": "L2",    # match your index metric
    "params": {"nprobe": 10}
}

#performs a semantic vector search using the query vector query_vectors
#searches within existing collection demo_collection and returns the top 2 similar results to the input query: query_vectors
res = milvus_client.search(
    collection_name="project_collection",
    data=query_vectors,
    anns_field="embedding",
    search_params = search_params,
    limit=2, #returns top 2 similar results
    output_fields=["text","FilePath","FileExtension"], #this is what you select -> in SQL: select text, subject from demo_collection where vector is similar to data

)

#2 relevant entries to the query are basically printed below
print(res)

data: [[{'id': 460142188850053121, 'distance': 1.0583128929138184, 'entity': {'FileExtension': '.webp', 'FilePath': 'Images/Samples/unicorn.webp', 'text': 'A majestic white unicorn with a golden horn stands gracefully in a mystical forest stream, surrounded by vibrant foliage and ancient trees.'}}, {'id': 460142207523618830, 'distance': 1.6614482402801514, 'entity': {'FileExtension': '.jpeg', 'FilePath': 'Text/Samples/trouble_is_my_business.txt', 'text': "The document describes a dark, hard-boiled film noir tale set in Los Angeles in 1947, where smog creates a pervasive haze mirroring the city's moral ambiguity. The protagonist is Roland Drake, a disgraced private eye who has lost his office and reputation due to a botched missing persons case. His career seems over until a seductive, dark-haired woman named Katherine Montemar walks in. She seeks his help finding her missing family, and the immediate chemistry between them leads to a romance. However, Drake wakes up to find Katherine g

In [ ]:
#performs a semantic vector search using the query vector query_vectors
#searches within existing collection demo_collection and returns the top 2 similar results to the input query: query_vectors

query_vectors1 = embedding_fn.encode_queries(["How do I make brownies?"])

res1 = milvus_client.search(
    collection_name="project_collection",
    data=query_vectors1,
    anns_field="embedding",
    search_params = search_params,
    limit=2, #returns top 2 similar results
    output_fields=["text","FilePath","FileExtension"], #this is what you select -> in SQL: select text, subject from demo_collection where vector is similar to data

)

#2 relevant entries to the query are basically printed below
print(res1)

data: [[{'id': 460142202164871176, 'distance': 1.556919813156128, 'entity': {'FileExtension': '.jpeg', 'FilePath': 'PDF/Samples/ClayMoldMaking.pdf', 'text': 'This document outlines an Arts Center Curriculum for "Mold Making," teaching students to create cast objects using either plaster (grades 3-12) or pewter (high school). The lesson is designed for a minimum of one hour.\n\n**Core Concepts:**\n*   Molds are prevalent in everyday life and art.\n*   Artists like Mary Nohl and Stella Waitzkin utilize molds in their work.\n*   Molds function as negative space, shaping the final object.\n\n**Learning Objectives:**\n*   Students will recognize the commonality of molds.\n*   They will design molds emphasizing depth and pattern.\n*   They will create a mold and cast an object.\n\n**Lesson Flow:**\n1.  **Discussion:** Brainstorm how molds are used daily, analyze examples from artists, and understand the concept of negative space in molds.\n2.  **Plaster Casting (Simpler Method):**\n    *   S

In [ ]:
from pprint import pprint

In [ ]:
#performs a semantic vector search using the query vector query_vectors
#searches within existing collection demo_collection and returns the top 2 similar results to the input query: query_vectors

query_vectors2 = embedding_fn.encode_queries(["Instructions on How to Dance"])

res2 = milvus_client.search(
    collection_name="project_collection",
    data=query_vectors2,
    anns_field="embedding",
    search_params = search_params,
    limit=1, #returns top 2 similar results
    output_fields=["text","FilePath","FileExtension"], #this is what you select -> in SQL: select text, subject from demo_collection where vector is similar to data

)

#2 relevant entries to the query are basically printed below
print(res2)
pprint(res2)

data: [[{'id': 460142202164871175, 'distance': 1.258986473083496, 'entity': {'FileExtension': '.webp', 'FilePath': 'PDF/Samples/DogTrainingBasics.pdf', 'text': 'This document is "The Basics of Training Your Dog: Leader Guide," a Wisconsin 4-H publication designed to assist 4-H leaders and members in dog obedience training.\n\nThe guide outlines a structured, step-by-step curriculum for teaching foundational dog obedience skills. It emphasizes key training principles such as patience, consistent daily practice (preferring two 15-minute sessions), immediate correction, and ample positive reinforcement (praise). It also highlights the importance of training in a class setting with other dogs to ensure the dog learns to obey in various environments, and stresses that dog training is a "team" effort between handler and dog. Proper collar placement high on the neck is crucial for control, and dogs should be at least nine months old and healthy to begin training.\n\nThe core curriculum is div

In [ ]:
#performs a semantic vector search using the query vector query_vectors
#searches within existing collection demo_collection and returns the top 2 similar results to the input query: query_vectors

query_vectors3 = embedding_fn.encode_queries(["Advice for training a golden retriever"])

res3 = milvus_client.search(
    collection_name="project_collection",
    data=query_vectors3,
    anns_field="embedding",
    search_params = search_params,
    limit=2, #returns top 2 similar results
    output_fields=["text","FilePath","FileExtension"], #this is what you select -> in SQL: select text, subject from demo_collection where vector is similar to data

)

#2 relevant entries to the query are basically printed below
print(res3)

data: [[{'id': 460142202164871175, 'distance': 1.030322551727295, 'entity': {'FileExtension': '.webp', 'FilePath': 'PDF/Samples/DogTrainingBasics.pdf', 'text': 'This document is "The Basics of Training Your Dog: Leader Guide," a Wisconsin 4-H publication designed to assist 4-H leaders and members in dog obedience training.\n\nThe guide outlines a structured, step-by-step curriculum for teaching foundational dog obedience skills. It emphasizes key training principles such as patience, consistent daily practice (preferring two 15-minute sessions), immediate correction, and ample positive reinforcement (praise). It also highlights the importance of training in a class setting with other dogs to ensure the dog learns to obey in various environments, and stresses that dog training is a "team" effort between handler and dog. Proper collar placement high on the neck is crucial for control, and dogs should be at least nine months old and healthy to begin training.\n\nThe core curriculum is div

In [ ]:
import os


#performs a semantic vector search using the query vector query_vectors
#searches within existing collection demo_collection and returns the top 2 similar results to the input query: query_vectors

query_vectors4 = embedding_fn.encode_queries(["Show me a house tour"])

res4 = milvus_client.search(
    collection_name="project_collection",
    data=query_vectors4,
    anns_field="embedding",
    search_params = search_params,
    limit=2, #returns top 2 similar results
    output_fields=["text","FilePath","FileExtension"], #this is what you select -> in SQL: select text, subject from demo_collection where vector is similar to data

)

#2 relevant entries to the query are basically printed below
print(res4)
innerRes4 = res4[0]

for dict in innerRes4:
  distance = dict.distance
  entityDict = dict.entity
  textSummary = entityDict.text
  FilePath = entityDict.FilePath
  FileExtension = entityDict.FileExtension
  print(distance)
  print("---------------------------")
  print(textSummary)
  print("---------------------------")
  print(FilePath)
  print("---------------------------")
  Filename = os.path.basename(FilePath)
  print(Filename)  # Output: House.mov
  print("---------------------------")
  print(FileExtension)

data: [[{'id': 460142216116961300, 'distance': 1.325439214706421, 'entity': {'FileExtension': '.jpeg', 'FilePath': 'Videos/Samples/House.mov', 'text': 'The video showcases a large, modern luxury house at dusk, with its numerous windows glowing warmly from the interior lights. The camera slowly pans across the exterior, revealing the contemporary architecture and the expansive facade of the multi-story building. The on-screen text identifies the property as an "incredible ultra luxury" home located in Frisco, Texas.'}}, {'id': 460142216116961299, 'distance': 1.465499758720398, 'entity': {'FileExtension': '.webp', 'FilePath': 'Videos/Samples/video.mp4', 'text': 'The video presents a static view of a vibrant park scene on a sunny day. Lush green trees with sunlight filtering through their leaves fill the right side of the frame, alongside a paved path and a wooden bench. On the left, a busy street features cars and a bus driving by.'}}]]
1.325439214706421
---------------------------
The v

In [ ]:
print("Query vector length:", len(query_vectors[0]))


Query vector length: 768
